In [1]:
import pandas as pd

In [15]:
dfA = pd.read_csv(r"C:\hw-8-Data_integration\Project\Record linkage\DATASETS\Dataset separati per lettera\x.csv")
dfA.head()

,ID,name,country,headquarters,address,founded,employees,market_cap,revenue,profit,industry,sector,categories,ceo;
0,360,xpressbees,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN,logistics,NaN,NaN,;
1,450,xl dynamics,NaN,NaN,NaN,2002,NaN,NaN,NaN,NaN,accounting & auditing,NaN,NaN,;
2,459,xoriant,NaN,NaN,NaN,1990,NaN,NaN,NaN,NaN,it services & consulting,NaN,NaN,;
3,617,xcelserv solutions,NaN,NaN,NaN,2011,NaN,NaN,NaN,NaN,it services & consulting,NaN,NaN,;
4,702,xiaomi,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,consumer electronics & appliances,NaN,NaN,;


In [16]:
df = pd.read_csv(r"C:\hw-8-Data_integration\Project\Record linkage\DATASETS\Predictions\x.csv")
df.head()

,match_score,ltable_ID,rtable_ID,ltable_name,ltable_country,ltable_headquarters,ltable_address,ltable_founded,ltable_employees,ltable_market_cap,...,rtable_address,rtable_founded,rtable_employees,rtable_market_cap,rtable_revenue,rtable_profit,rtable_industry,rtable_sector,rtable_categories,rtable_ceo
0,0.992537,360,360,xpressbees,NaN,NaN,NaN,2015,NaN,NaN,...,NaN,2015,NaN,NaN,NaN,NaN,logistics,NaN,NaN,;
1,0.990072,450,450,xl dynamics,NaN,NaN,NaN,2002,NaN,NaN,...,NaN,2002,NaN,NaN,NaN,NaN,accounting & auditing,NaN,NaN,;
2,0.993665,459,459,xoriant,NaN,NaN,NaN,1990,NaN,NaN,...,NaN,1990,NaN,NaN,NaN,NaN,it services & consulting,NaN,NaN,;
3,0.979853,617,617,xcelserv solutions,NaN,NaN,NaN,2011,NaN,NaN,...,NaN,2011,NaN,NaN,NaN,NaN,it services & consulting,NaN,NaN,;
4,0.158793,4405,617,xylem water solutions,NaN,NaN,NaN,2011,NaN,NaN,...,NaN,2011,NaN,NaN,NaN,NaN,it services & consulting,NaN,NaN,;


In [30]:
threshold = 0.5
dict_match = {}
df = df.reset_index()
for index, row in df.iterrows():
    if row["match_score"] > threshold:
        score = row["match_score"]
        l_id = row['ltable_ID']
        r_id = row['rtable_ID']
        if l_id not in dict_match.keys():
            dict_match[l_id] = set()
        dict_match[l_id].add(r_id) #((r_id, score))
        if r_id not in dict_match.keys():
            dict_match[r_id] = set()
        dict_match[r_id].add(l_id) #((l_id, score))

In [31]:
final_tuple_match = []
for key, values in dict_match.items():
    ok = False
    for sublist in final_tuple_match:
        if key in sublist:
            sublist.update(values)
            ok = True
    if not ok:
        new_set = set(values)
        final_tuple_match.append(new_set)

### Caso 1

In [9]:
def dict_list_to_dict_first_value(d):
    return {k: v[0] for k, v in d.items() if len(v) >= 1}

In [14]:
data_merged = []
for values in final_tuple_match:
    tmp_dict = {}
    for value in values:
        tmp_dict.update(dict_list_to_dict_first_value(dfA.loc[dfA['id'] == value].to_dict(orient="list")))
    data_merged.append(tmp_dict)

In [15]:
df = pd.DataFrame(data_merged)
df

,id,name,country,revenue_2022,profit,assets,market_value
0,9,toyota motor,japan,doll_281.75 b,NaN,NaN,doll_237.73 b
1,21,unitedhealth group,united states,doll_297.55 b,NaN,NaN,doll_490.15 b
2,27,tencent holdings,china,doll_86.86 b,NaN,NaN,doll_414.28 b
3,40,mercedes-benz group,germany,doll_178.94 b,NaN,NaN,doll_74.62 b
4,41,cvs health,united states,doll_291.98 b,NaN,NaN,doll_133.53 b
...,...,...,...,...,...,...,...
209,992,link reit,hong kong,doll_1.47 b,NaN,NaN,doll_18.28 b
210,993,yum! brands,canada,doll_6.58 b,NaN,NaN,doll_35.28 b
211,995,aercap holdings,ireland,doll_4.52 b,NaN,NaN,doll_12.59 b
212,898,china merchants securities,china,doll_5.7 b,NaN,NaN,doll_9.86 b


### Caso 2

In [20]:
def most_frequent(l):
    return max(set(l), key = l.count)

In [33]:
def merge_list_of_dict(df, values):
    keys = list(df.columns)
    dict_of_merge = {k: [] for k in keys}
    for value in values:
        tmp = dict_list_to_dict_first_value(df.loc[df['ID'] == value].to_dict(orient="list"))
        for k, v in tmp.items():
            dict_of_merge[k].append(v)
    return {k: most_frequent(v) for k, v in dict_of_merge.items()}

In [34]:
res = []
for values in final_tuple_match:
    res.append(merge_list_of_dict(dfA, values))

In [35]:
df = pd.DataFrame(res)
df

,ID,name,country,headquarters,address,founded,employees,market_cap,revenue,profit,industry,sector,categories,ceo;
0,360,xpressbees,NaN,NaN,NaN,2015,NaN,NaN,NaN,NaN,logistics,NaN,NaN,;
1,450,xl dynamics,NaN,NaN,NaN,2002,NaN,NaN,NaN,NaN,accounting & auditing,NaN,NaN,;
2,459,xoriant,NaN,NaN,NaN,1990,NaN,NaN,NaN,NaN,it services & consulting,NaN,NaN,;
3,617,xcelserv solutions,NaN,NaN,NaN,2006,NaN,NaN,NaN,NaN,it services & consulting,NaN,NaN,;
4,22689,xiaomi,china,NaN,NaN,NaN,NaN,doll_37.76 b,NaN,NaN,NaN,NaN,NaN,;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,146431,xiamen goldenhome,china,NaN,NaN,NaN,NaN,doll_0.73 m,NaN,NaN,NaN,NaN,NaN,;
162,152117,xinte energy,china,NaN,NaN,NaN,NaN,doll_2.63 b,NaN,0.0,NaN,NaN,NaN,;
163,183550,xlab d.o.o,NaN,ljubljana,NaN,2001,NaN,NaN,NaN,NaN,technology,software,NaN,;
164,181330,xhamster,NaN,nicosia,NaN,2007,NaN,NaN,NaN,NaN,technology,software,NaN,;
